## Mixed-Curvature SNE

### Re-create t-SNE visualization on MNIST

### Re-create CO-SNE visualization on synthetic point clusters

### Create hyperspherical data from wrapped normal distribution and generate visualization

### Create S^2 x E^2 x H^2 product space from wrapped normal distribution, and generate three separate 2D visualizations